In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
circle_dir = '/content/drive/My Drive/data/circle'
square_dir = '/content/drive/My Drive/data/square'
triangle_dir = '/content/drive/My Drive/data/triangle'
test_dir = '/content/drive/My Drive/data/test'

In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [8]:
import pandas as pd



circle = os.listdir(circle_dir)
square = os.listdir(square_dir)
triangle = os.listdir(triangle_dir)
test = os.listdir(test_dir)


In [9]:
df=pd.DataFrame({'filename': circle + square + triangle,
                 'label': ['circle'] * len(circle) + ['square'] * len(square) + ['triangle'] * len(triangle),
                 })

print(df.head())

   filename   label
0  1558.png  circle
1  2559.png  circle
2  1256.png  circle
3  1828.png  circle
4  1964.png  circle


In [10]:
from tensorflow.keras.preprocessing import image
import random

image_aleatoire = random.choice(circle)



img_path = os.path.join(circle_dir, image_aleatoire)
img_path

'/content/drive/My Drive/data/circle/3398.png'

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [12]:

circle_df = pd.DataFrame({
    'filename': ['circle/' + f for f in circle],
    'label': 'circle'
}).sample(n=1500, random_state=42)

square_df = pd.DataFrame({

    'filename': ['square/' + f for f in square],
    'label': 'square'
}).sample(n=1500, random_state=42)

triangle_df = pd.DataFrame({

    'filename': ['triangle/' + f for f in triangle],
    'label': 'triangle'
}).sample(n=1500, random_state=42)

df_final = pd.concat([circle_df, square_df,triangle_df]).reset_index(drop=True)
df_final

,filename,label
0,circle/2215.png,circle
1,circle/289.png,circle
2,circle/462.png,circle
3,circle/2142.png,circle
4,circle/2861.png,circle
...,...,...
4495,triangle/1454.png,triangle
4496,triangle/3403.png,triangle
4497,triangle/3220.png,triangle
4498,triangle/368.png,triangle


In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)
df_final

,filename,label
0,circle/2215.png,circle
1,circle/289.png,circle
2,circle/462.png,circle
3,circle/2142.png,circle
4,circle/2861.png,circle
...,...,...
4495,triangle/1454.png,triangle
4496,triangle/3403.png,triangle
4497,triangle/3220.png,triangle
4498,triangle/368.png,triangle


In [14]:
df_final['label'] = df_final['label'].astype(str)


train_generator = train_datagen.flow_from_dataframe(
    df_final,
    "/content/drive/My Drive/data",
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_dataframe(
    dataframe=df_final,
    directory="/content/drive/My Drive/data",
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 3600 validated image filenames belonging to 3 classes.
Found 900 validated image filenames belonging to 3 classes.


In [15]:
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models


model = models.Sequential()


model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))  # Couche de convolution : 32 filtres de 3x3, détecte des motifs simples dans l'image, ReLU pour rendre le modèle non-linéaire, forme d'entrée (150x150x3).

model.add(layers.MaxPooling2D((2, 2)))  # Couche de max pooling : réduit les dimensions de l'image en prenant le maximum de chaque carré de 2x2 pixels.

model.add(layers.Conv2D(64, (3, 3), activation='relu'))  # Deuxième couche de convolution : 64 filtres de 3x3, détecte des motifs plus complexes, ReLU pour l'activation.

model.add(layers.MaxPooling2D((2, 2)))  # Deuxième couche de max pooling : réduit encore les dimensions de l'image.

model.add(layers.Conv2D(128, (3, 3), activation='relu'))  # Troisième couche de convolution : 128 filtres de 3x3, détecte des motifs encore plus complexes, ReLU pour l'activation.

model.add(layers.MaxPooling2D((2, 2)))  # Troisième couche de max pooling : réduit encore les dimensions de l'image.

model.add(layers.Flatten())  # Couche Flatten : transforme les données 2D en un vecteur 1D pour la couche dense suivante.

model.add(layers.Dense(512, activation='relu'))  # Couche dense (fully connected) : 512 neurones, chaque neurone est connecté à tous les neurones de la couche précédente, ReLU pour l'activation.

model.add(layers.Dense(3, activation='softmax'))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
from keras import optimizers


optimi = optimizers.RMSprop(learning_rate=1e-4)
model.compile(optimizer=optimi,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
# Entraînement du modèle et sauvegarde de l'historique d'entraînement dans 'history'
history = model.fit(
    train_generator,  # Utilise le générateur pour les données d'entraînement

    # Le nombre d'étapes (batches) à exécuter à chaque époque.
    # Calculé comme le nombre total d'échantillons d'entraînement divisé par la taille du lot.
    steps_per_epoch=train_generator.samples // train_generator.batch_size,

    epochs=5,  # Nombre total d'époques pour l'entraînement

    validation_data=validation_generator,  # Utilise le générateur pour les données de validation

    # Le nombre d'étapes (batches) à exécuter pour chaque époque de validation.
    # Calculé comme le nombre total d'échantillons de validation divisé par la taille du lot.
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 1863s 16s/step - accuracy: 0.7914 - loss: 0.5955 - val_accuracy: 0.9944 - val_loss: 0.0376
Epoch 2/5
  1/112 ━━━━━━━━━━━━━━━━━━━━ 2:39 1s/step - accuracy: 1.0000 - loss: 0.0617

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


112/112 ━━━━━━━━━━━━━━━━━━━━ 16s 132ms/step - accuracy: 1.0000 - loss: 0.0617 - val_accuracy: 0.8627 - val_loss: 0.3139
Epoch 3/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.9940 - loss: 0.0304 - val_accuracy: 0.9911 - val_loss: 0.0541
Epoch 4/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 20s 172ms/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 0.9911 - val_loss: 0.0549
Epoch 5/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.9967 - loss: 0.0118 - val_accuracy: 0.9933 - val_loss: 0.0369


In [18]:
validation_steps = validation_generator.samples // validation_generator.batch_size
evaluation = model.evaluate(validation_generator, steps=validation_steps)


print(f"Perte sur la validation: {evaluation[0]}")
print(f"Précision sur la validation: {evaluation[1]}")

28/28 ━━━━━━━━━━━━━━━━━━━━ 14s 510ms/step - accuracy: 0.9935 - loss: 0.0309
Perte sur la validation: 0.03682870417833328
Précision sur la validation: 0.9933035969734192


In [19]:
from keras.models import load_model


save_path = '/content/drive/My Drive/data/shape_model.h5'


model.save(save_path)

print(f"Modèle sauvegardé à l'emplacement : {save_path}")

Modèle sauvegardé à l'emplacement : /content/drive/My Drive/data/shape_model.h5


In [20]:
from keras.utils import load_img, img_to_array


model = load_model('/content/drive/My Drive/data/shape_model.h5')

circle_df = pd.DataFrame({
    'filename': ['circle/' + f for f in circle],
    'label': 'circle'
}).sample(n=10, random_state=42)

square_df = pd.DataFrame({
    'filename': ['square/' + f for f in square],
    'label': 'square'
}).sample(n=10, random_state=42)

triangle_df = pd.DataFrame({
    'filename': ['triangle/' + f for f in triangle],
    'label': 'triangle'
}).sample(n=10, random_state=42)


test_df = pd.DataFrame({
    'filename': test,

}).sample(n=10, random_state=42)


df_final = pd.concat([circle_df, square_df,triangle_df]).reset_index(drop=True)
df_final
df_final['label'] = df_final['label'].astype(str)



In [24]:

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    df_final,
    directory="/content/drive/My Drive/data",
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=10,
    class_mode='categorical',
    shuffle=False
)


predict_datagen = ImageDataGenerator(rescale=1./255)
predict_generator = predict_datagen.flow_from_dataframe(
    test_df,

    directory="/content/drive/My Drive/data/test",
    x_col='filename',

    target_size=(150, 150),
    batch_size=10,
    class_mode=None,
    shuffle=False
)


predictions = model.predict(test_generator)


Found 30 validated image filenames belonging to 3 classes.
Found 10 validated image filenames.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 215ms/step


In [25]:
import numpy as np
predicted_classes_indices = np.argmax(predictions, axis=1)
class_labels = list(test_generator.class_indices.keys())




predicted_classes = [class_labels[i] for i in predicted_classes_indices]


df_final['predictions'] = predicted_classes

df_final

,filename,label,predictions
0,circle/2215.png,circle,circle
1,circle/289.png,circle,circle
2,circle/462.png,circle,circle
3,circle/2142.png,circle,circle
4,circle/2861.png,circle,circle
5,circle/650.png,circle,circle
6,circle/508.png,circle,circle
7,circle/889.png,circle,circle
8,circle/1676.png,circle,circle
9,circle/1875.png,circle,circle


In [26]:

predictions_predict = model.predict(predict_generator, steps=len(predict_generator))


predicted_classes_indices_predict = np.argmax(predictions_predict, axis=1)


predicted_classes_predict = [class_labels[i] for i in predicted_classes_indices_predict]


test_df['predictions'] = predicted_classes_predict

print("\n--- Résultats de la prédiction (nouvelles données) ---")
print(test_df.head())

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step

--- Résultats de la prédiction (nouvelles données) ---
    filename predictions
33  3501.png      square
36  2705.png    triangle
4    264.png      circle
13  2696.png    triangle
30  3499.png      square
